In [1]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple

In [2]:
#pip install pdfplumber

In [3]:
#Line = namedtuple('Line',['v','m'])

In [4]:
Line = namedtuple('Line', 'college_code college_name branch_code branch_name status_category open_percentile')

In [5]:
college_re = re.compile(r'^(\d{4})\s-\s(.*)')

In [6]:
college_re.search('1002 - Government College of Engineering, Amravati').group(1)

'1002'

In [7]:
college_re.search('1002 - Government College of Engineering, Amravati').group(2)

'Government College of Engineering, Amravati'

In [8]:
branch_re = re.compile(r'(\d{9})\s-\s(.*)')

In [9]:
branch_re.search('100219110 - Civil Engineering').group(2)

'Civil Engineering'

In [10]:
status_re = re.compile(r'Status:\s(.*)')

In [11]:
status_re.search('Status: Government Autonomous').group(1)

'Government Autonomous'

In [12]:
open_percentile_re = re.compile(r'\(([0-9]*)\.([0-9]*)\)')

In [13]:
open_percentile_re.search('(85.7691222)')

<re.Match object; span=(0, 12), match='(85.7691222)'>

In [14]:
college1 = Line('1002','Government College of Engineering, Amravati','100219110','Civil Engineering','Government Autonomous','(85.7691222)')

In [15]:
college1.open_percentile


'(85.7691222)'

In [16]:
file = 'mhtcet.pdf'

In [17]:
file

'mhtcet.pdf'

In [18]:
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
            clg = college_re.search(line)
            branch = branch_re.search(line)
            status = status_re.search(line)
            open_percent = open_percentile_re.search(line)
            if clg:
                college_id, college_name = clg.group(1), clg.group(2)
            elif branch:
                branch_id, branch_name = branch.group(1), branch.group(2)
            elif status:
                status_category = status.group(1)
            elif open_percent:
                open_percentile = open_percent.group(1)+'.'+open_percent.group(2)
                lines.append(Line(college_id, college_name, branch_id, branch_name, status_category, open_percentile))

In [19]:
clg

In [20]:
text

"D Government of Maharashtra\ni State Common Entrance Test Cell\nr Cut Off List of All India Seats CAP Round I for Admission to First Year of Four Year Full Time Degree Courses in Engineering and Technology for the Academic Year 2021-22\n6938 - Shree Siddheshwar Women's College Of Engineering Solapur.\n693892550F - Computer Science and Engineering (Artificial Intelligence and Data Science)\nStatus: Un-Aided\nHome University Seats Allotted to Home University Candidates\nStage LOPENH\n  I 97541\n(6.5584481)\nOther Than Home University Seats Allotted to Other Than Home University Candidates\nStage LOPENO\n  I 62379\n(52.2112775)\nOther Than Home University Seats Allotted to Home University Candidates\nStage LOPENO\n  I 96596\n(8.1817268)\nState Level\nStage EWS\n  I 77392\n(36.4202981)\nCut Off Indicates All India Merit No.; Figures in bracket Indicates JEE(Main) Score/MHT-CET Percentile.; AI to AI-All India Seats Allotted to All India Candidature Candidates.\n1"

In [21]:
lines[:5]

[Line(college_code='1002', college_name='Government College of Engineering, Amravati', branch_code='100219110', branch_name='Civil Engineering', status_category='Government Autonomous', open_percentile='85.7691222'),
 Line(college_code='1002', college_name='Government College of Engineering, Amravati', branch_code='100224210', branch_name='Computer Science and Engineering', status_category='Government Autonomous', open_percentile='96.8328566'),
 Line(college_code='1002', college_name='Government College of Engineering, Amravati', branch_code='100224610', branch_name='Information Technology', status_category='Government Autonomous', open_percentile='95.3951022'),
 Line(college_code='1002', college_name='Government College of Engineering, Amravati', branch_code='100229310', branch_name='Electrical Engineering', status_category='Government Autonomous', open_percentile='87.9438958'),
 Line(college_code='1002', college_name='Government College of Engineering, Amravati', branch_code='1002372

In [22]:
df = pd.DataFrame(lines)

In [23]:
df.head()

,college_code,college_name,branch_code,branch_name,status_category,open_percentile
0,1002,"Government College of Engineering, Amravati",100219110,Civil Engineering,Government Autonomous,85.7691222
1,1002,"Government College of Engineering, Amravati",100224210,Computer Science and Engineering,Government Autonomous,96.8328566
2,1002,"Government College of Engineering, Amravati",100224610,Information Technology,Government Autonomous,95.3951022
3,1002,"Government College of Engineering, Amravati",100229310,Electrical Engineering,Government Autonomous,87.9438958
4,1002,"Government College of Engineering, Amravati",100237210,Electronics and Telecommunication Engg,Government Autonomous,92.2042446


In [24]:
df.to_csv('mhtcet.csv',index=False)